In [13]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split, SubsetRandomSampler
import numpy as np
import os

In [14]:
data_dir = 'data\human detection dataset'

### Split dataset

In [15]:
transform = transforms.Compose([
    transforms.RandomResizedCrop(64),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(30),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
]) #define transform

In [16]:
dataset = datasets.ImageFolder(root=data_dir, transform=transform)
batch_size = 64

In [17]:
validation_split = 0.2
dataset_size = len(dataset)
val_size = int(validation_split * dataset_size)
train_size = dataset_size - val_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [18]:
print("Number of Image in Train Dataset:", len(train_loader))
print("Number of Image in Validation Dataset:", len(val_loader))

Number of Image in Train Dataset: 12
Number of Image in Validation Dataset: 3


In [19]:
import torch.nn as nn
import torch.optim as optim

In [20]:
import torchvision.models as models

class ResNet50(nn.Module):
    def __init__(self, num_classes=2):
        super(ResNet50, self).__init__()
        resnet50 = models.resnet50(pretrained=True)
        modules = list(resnet50.children())[:-1]
        self.resnet50 = nn.Sequential(*modules)
        self.fc = nn.Sequential(
            nn.Linear(resnet50.fc.in_features, 1024),
            nn.ReLU(),
            nn.BatchNorm1d(1024),
            nn.Linear(1024, num_classes)
        )

    def forward(self, x):
        x = self.resnet50(x)
        x = x.mean([2, 3])
        x = self.fc(x)
        return x


In [21]:
model = ResNet50(num_classes=2)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

C:\Users\ching\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\ching\AppData\Roaming\Python\Python311\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [22]:
if torch.cuda.is_available():
    device = 'cuda'
    model.to(device)

In [24]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(train_loader)}')

model.eval()
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in val_loader:
        inputs = inputs.to(device)
        labels = labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy on validation set: {correct / total}')

Epoch 1/10, Loss: 0.5248360509673754
Epoch 2/10, Loss: 0.4953499337037404
Epoch 3/10, Loss: 0.49325336267550784
Epoch 4/10, Loss: 0.47783851871887845
Epoch 5/10, Loss: 0.47000568360090256
Epoch 6/10, Loss: 0.43198824177185696
Epoch 7/10, Loss: 0.5230276311437289
Epoch 8/10, Loss: 0.5098868409792582
Epoch 9/10, Loss: 0.47303881744543713
Epoch 10/10, Loss: 0.4594479774435361
Accuracy on validation set: 0.7989130434782609


In [25]:
torch.save(model.state_dict(), 'model.pth')